
# First - get team_id       

url = http://mlb.mlb.com/lookup/json/named.team_hitting_season_leader_master.bam?season=2016&sort_order=%27desc%27&sort_column=%27avg%27&game_type=%27R%27&sport_code=%27mlb%27&recSP=1&recPP=50

# Second - get player_id

url = http://mlb.mlb.com/pubajax/wf/flow/stats.splayer?season=2016&sort_order=%27desc%27&sort_column=%27avg%27&stat_type=hitting&page_type=SortablePlayer&team_id=112&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50



# Third - use player_id to get gamelog

url = http://m.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id=493128&season=2016&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27


# mlb_crawler(stat_type,season)
    
    stat_type = hitting,pitching,fielding(?)
    season = max is lastest season

In [43]:
import os
import csv
import requests as r
import json

#start,get dataType(hitting,pitching,fieding(?)),teamID
def mlb_crawler(statType,season):
    
    for season in range(season,1999,-1):
        
        season_path = '../MLB_DATA/{}'.format(season)
        if not os.path.exists(season_path):                   #check dir is already existx or not
            os.makedirs(season_path)

        toGetTeamID = r.get('http://mlb.mlb.com/lookup/json/named.team_{}_season_leader_master.bam?season={}&sort_order=%27desc%27&sort_column=%27avg%27&game_type=%27R%27&sport_code=%27mlb%27&recSP=1&recPP=50'.format(statType,season))
        teamSeasonStat = json.loads(toGetTeamID.text)['team_{}_season_leader_master'.format(statType)]['queryResults']['row']

        team={}
        for team_tag in teamSeasonStat:
            team[team_tag['team_id']] = team_tag['team_full']

        for team_id in team:
            setSeasonAndTeamID(season,team[team_id],team_id,statType,season_path) #setSeasonAndTeamID(season,team_name,team_id) #season max value=2016

#set season and team_id
def setSeasonAndTeamID(season,team_name,team_id,statType,season_path):
    
    team_path = season_path + '/{}'.format(team_name)          #set data storage path
    if not os.path.exists(team_path):                       #check dir is already existx or not
        os.makedirs(team_path)
                    
    getPlayerID(season,team_name,team_id,team_path,statType)
        
#to get every team main-40 player_id        
def getPlayerID(season,team_name,team_id,team_path,statType):
    
    resToGetPlayerID = r.get('http://mlb.mlb.com/pubajax/wf/flow/stats.splayer?season={}&sort_order=%27desc%27&sort_column=%27avg%27&stat_type={}&page_type=SortablePlayer&team_id={}&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50'.format(season,statType,team_id))
    playerStats = json.loads(resToGetPlayerID.text)['stats_sortable_player']['queryResults']['row']

    player_ids=[]
    playerIdName={}
    for playerID in playerStats:
        player_id = playerID['player_id']
        playerIdName[playerID['player_id']] = playerID['name_display_first_last']
        
        player_ids.append(player_id)                        #collect all every player_id by team
    #print(team_name + "--- {}".format(len(player_ids)))
    for id in player_ids:
        getPlayerGameLog(id,playerIdName[id],season,team_name,team_path,statType)

#to get every player gamelog
def getPlayerGameLog(id,playerName,season,team_name,team_path,statType):
    
    resToGetGameLog = r.get('http://m.mlb.com/lookup/json/named.sport_{}_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id={}&season={}&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27'.format(statType,id,season))
    gameLog = json.loads(resToGetGameLog.text)['sport_{}_game_log_composed'.format(statType)]['sport_{}_game_log'.format(statType)]['queryResults']['row']
            
    try:  #deal keyError with player who have no data
        
        fieldnames = list((gameLog[0]).keys())
        with open(team_path + '/{}_{}_{}GameLog.csv'.format(season,team_name,statType),'a') as csvfile:

            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in gameLog:
                writer.writerow(row)
                
    except KeyError:
        with open(team_path + '/{}_except_player.txt'.format(statType),'a') as text_file:
            #print(team_name +':'+ str(season) + ':' + id + ' : ' + playerName)
            text_file.write(team_name +':'+ str(season) + ':' + id + ':' + playerName)


In [41]:
mlb_crawler('hitting',2016)  #dataType,season(max = latest season)

 



KeyboardInterrupt: 